# NASA C-MAPSS Turbofan Engine Degradation Analysis

Analyse du dataset NASA pour la prediction de la duree de vie utile restante (RUL) des moteurs turbofan.

## Structure des donnees

- **unit_id**: Identifiant du moteur
- **cycle**: Cycle de fonctionnement
- **op_setting_1, 2, 3**: Parametres operationnels
- **sensor_1 to sensor_21**: Mesures des 21 capteurs

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

sns.set_theme(style="whitegrid")
pd.set_option('display.max_columns', 30)

## 1. Chargement des donnees

In [ ]:
# Noms des colonnes
columns = ['unit_id', 'cycle', 'op_setting_1', 'op_setting_2', 'op_setting_3'] + \
          [f'sensor_{i}' for i in range(1, 22)]

# Charger le dataset FD001 (plus simple pour commencer)
train_df = pd.read_csv('data/CMaps/train_FD001.txt', sep='\s+', header=None, names=columns)
test_df = pd.read_csv('data/CMaps/test_FD001.txt', sep='\s+', header=None, names=columns)
rul_df = pd.read_csv('data/CMaps/RUL_FD001.txt', sep='\s+', header=None, names=['RUL'])

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")
print(f"RUL shape: {rul_df.shape}")

In [ ]:
train_df.head(10)

In [ ]:
train_df.describe()

## 2. Calcul du RUL (Remaining Useful Life)

Pour les donnees d'entrainement, le RUL est calcule comme:
**RUL = max_cycle - current_cycle**

In [ ]:
# Calculer le RUL pour chaque observation dans le train set
def calculate_rul(df):
    # Trouver le cycle max pour chaque moteur
    max_cycles = df.groupby('unit_id')['cycle'].max().reset_index()
    max_cycles.columns = ['unit_id', 'max_cycle']
    
    # Joindre et calculer le RUL
    df = df.merge(max_cycles, on='unit_id')
    df['RUL'] = df['max_cycle'] - df['cycle']
    df.drop('max_cycle', axis=1, inplace=True)
    
    return df

train_df = calculate_rul(train_df)
print(f"Nombre de moteurs: {train_df['unit_id'].nunique()}")
print(f"\nDistribution du RUL:")
print(train_df['RUL'].describe())

## 3. Exploration des donnees

In [ ]:
# Distribution de la duree de vie des moteurs
engine_life = train_df.groupby('unit_id')['cycle'].max()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(engine_life, bins=30, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Duree de vie (cycles)')
axes[0].set_ylabel('Nombre de moteurs')
axes[0].set_title('Distribution de la duree de vie des moteurs')
axes[0].axvline(engine_life.mean(), color='red', linestyle='--', label=f'Moyenne: {engine_life.mean():.0f}')
axes[0].legend()

axes[1].boxplot(engine_life)
axes[1].set_ylabel('Duree de vie (cycles)')
axes[1].set_title('Boxplot de la duree de vie')

plt.tight_layout()
plt.show()

print(f"Duree de vie moyenne: {engine_life.mean():.1f} cycles")
print(f"Duree de vie min: {engine_life.min()} cycles")
print(f"Duree de vie max: {engine_life.max()} cycles")

In [ ]:
# Verifier les valeurs manquantes
print("Valeurs manquantes par colonne:")
print(train_df.isnull().sum().sum())

## 4. Analyse des capteurs

In [ ]:
# Variance des capteurs - identifier ceux qui ont peu de variation
sensor_cols = [f'sensor_{i}' for i in range(1, 22)]
sensor_variance = train_df[sensor_cols].var().sort_values(ascending=False)

plt.figure(figsize=(12, 6))
sensor_variance.plot(kind='bar')
plt.title('Variance des capteurs')
plt.xlabel('Capteur')
plt.ylabel('Variance')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Capteurs avec variance proche de zero (peu utiles)
low_variance = sensor_variance[sensor_variance < 0.0001]
print(f"\nCapteurs avec variance quasi-nulle: {list(low_variance.index)}")

In [ ]:
# Correlation des capteurs avec le RUL
correlation_with_rul = train_df[sensor_cols + ['RUL']].corr()['RUL'].drop('RUL').sort_values()

plt.figure(figsize=(12, 6))
correlation_with_rul.plot(kind='barh', color=['red' if x < 0 else 'green' for x in correlation_with_rul])
plt.title('Correlation des capteurs avec le RUL')
plt.xlabel('Coefficient de correlation')
plt.tight_layout()
plt.show()

print("\nTop 5 capteurs les plus correles (positivement):")
print(correlation_with_rul.tail(5))
print("\nTop 5 capteurs les plus correles (negativement):")
print(correlation_with_rul.head(5))

In [ ]:
# Evolution des capteurs pour un moteur specifique
engine_id = 1
engine_data = train_df[train_df['unit_id'] == engine_id]

# Selectionner les capteurs les plus correles avec le RUL
top_sensors = correlation_with_rul.abs().sort_values(ascending=False).head(6).index.tolist()

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, sensor in enumerate(top_sensors):
    axes[i].plot(engine_data['cycle'], engine_data[sensor])
    axes[i].set_xlabel('Cycle')
    axes[i].set_ylabel(sensor)
    axes[i].set_title(f'{sensor} - Moteur {engine_id}')

plt.suptitle('Evolution des capteurs les plus correles avec le RUL', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Comparer l'evolution de plusieurs moteurs
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

sensors_to_plot = ['sensor_2', 'sensor_4', 'sensor_11', 'sensor_15']
engines_to_plot = [1, 10, 50, 80]

for idx, sensor in enumerate(sensors_to_plot):
    ax = axes[idx // 2, idx % 2]
    for engine in engines_to_plot:
        data = train_df[train_df['unit_id'] == engine]
        ax.plot(data['cycle'], data[sensor], label=f'Moteur {engine}', alpha=0.7)
    ax.set_xlabel('Cycle')
    ax.set_ylabel(sensor)
    ax.set_title(f'Evolution de {sensor}')
    ax.legend()

plt.tight_layout()
plt.show()